In [18]:
import pyWinhook as pyHook
import pythoncom
import pandas as pd
import numpy as np
import csv
import PySimpleGUI as sg

In [19]:
# make a dataframe
df = pd.DataFrame(columns=['Key', 'Ascii', 'Time'])
userFilePath = 'myInputCsv.csv'

In [20]:

class KeyLogger:
    def __init__(self):
        self.enterPressed = False
        self.eventList = []
        self.isCaps = False
        self.userName = getUserName()
        #self.message = ""

    def keyDownEvent(self, event):
        if event.KeyID == 20 and self.isCaps == False:
            self.isCaps = True
        elif event.KeyID == 20 and self.isCaps == True:
            self.isCaps = False     
        if event.KeyID>= 48 and event.KeyID<=57:
            event.Ascii = event.KeyID
        if self.isCaps == True and event.Ascii>=97 and event.Ascii<=122:
            event.Ascii = event.KeyID
        self.storeEvent("Down", event) 
        return True
    
    def mainLoop(self):
        while not self.enterPressed:
            pythoncom.PumpWaitingMessages()

            
    def keyUpEvent(self, event):
        if event.KeyID>= 48 and event.KeyID<=57:
            event.Ascii = event.KeyID
        if self.isCaps == True and event.Ascii>=97 and event.Ascii<=122:
            event.Ascii = event.KeyID
        print(chr(event.Ascii),end='')
        self.storeEvent("Up", event)
        return True
    
    def storeEvent(self, activity, event):
        keystrokeTime = int(event.Time)
        self.eventList.append ((self.userName,event.Ascii,event.Key,activity, int(keystrokeTime)))

        # Chosen to use Escape key (ESC) due to input using a similar method
        # Enter Key - KeyCode: 13 Ascii: 13 ScanCode: 28 - ESC = 27 @ Ascii
        # Activity condition had to be added because the userRecordData function was being called twice because of the keyUpEvent and keyDownEvent
        if event.Ascii == 13 and activity == "Down": 
            self.enterPressed = True
            print("Enter Pressed")
            userRecordData(self.eventList)

def userRecordData(eventList):
    print("\nOutput:")
    print(eventList)
    with open(userFilePath,'a',newline='\n') as f:
        writer = csv.writer(f)
        writer.writerows(eventList)
    f.close()

def getUserName():
    userName = input("Enter your name: ")
    return userName


#Not needed currently
def getKeyStroke():
    
    keyLogger = KeyLogger()
    hookManager = pyHook.HookManager()
    hookManager.KeyDown = keyLogger.keyDownEvent
    hookManager.KeyUp = keyLogger.keyUpEvent
    hookManager.HookKeyboard()

    keyLogger.mainLoop()
    # Unhooks the keyboard, no more data recorded, returns to menu
    hookManager.UnhookKeyboard()


In [21]:
sg.theme('DarkAmber')   # Add a touch of color

#TODO: 1) Make it so when the username is entered the Up event of the enter key is not recorded
#  2) Make it so the csv file is cretated only when an arbitrary number of passwords have been entered
#  3) Make it so the csv file is created when the user presses a button called "Make csv"


keyLogger = KeyLogger()
hookManager = pyHook.HookManager()
hookManager.KeyDown = keyLogger.keyDownEvent
hookManager.KeyUp = keyLogger.keyUpEvent
hookManager.HookKeyboard()

# All the stuff inside your window.
layout = [  [sg.Text('Collecting KeyStroke Data for User: ' + keyLogger.userName)],
            [sg.Text('Enter password (.tie5Roanl)'), sg.InputText( key='Input1')],
            [sg.Button('Ok'), sg.Button('Cancel'), sg.Push(), sg.Button('Make csv')] ]

# Create the Window
window = sg.Window('Window Title', layout, finalize=True)

# Bind the Enter key to the input field -> Used to clear the input field when enter is pressed
# For this to work, when creating the window, finalize must be set to True
window['Input1'].bind('<Return>', '_Enter')

# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Cancel': # if user closes window or clicks cancel
        break
    elif event == 'Input1' + '_Enter':
        window['Input1'].update('')
    
    #keyLogger.mainLoop()
    pythoncom.PumpWaitingMessages() # This is needed for the keylogger to work, and works without the mainLoop function

    #print('You entered ', values[0])
    print(values)

# Unhooks the keyboard, no more data recorded, returns to menu
hookManager.UnhookKeyboard()
window.close()

.tie5 roanlEnter Pressed

Output:
[('vuko', 13, 'Return', 'Up', 700940375), ('vuko', 46, 'Oem_Period', 'Down', 700944046), ('vuko', 46, 'Oem_Period', 'Up', 700944125), ('vuko', 116, 'T', 'Down', 700944343), ('vuko', 116, 'T', 'Up', 700944421), ('vuko', 105, 'I', 'Down', 700944421), ('vuko', 105, 'I', 'Up', 700944484), ('vuko', 101, 'E', 'Down', 700944578), ('vuko', 101, 'E', 'Up', 700944640), ('vuko', 53, '5', 'Down', 700945359), ('vuko', 53, '5', 'Up', 700945421), ('vuko', 0, 'Lshift', 'Down', 700945656), ('vuko', 82, 'R', 'Down', 700945812), ('vuko', 0, 'Lshift', 'Up', 700945875), ('vuko', 114, 'R', 'Up', 700945890), ('vuko', 111, 'O', 'Down', 700946015), ('vuko', 111, 'O', 'Up', 700946078), ('vuko', 97, 'A', 'Down', 700946140), ('vuko', 97, 'A', 'Up', 700946234), ('vuko', 110, 'N', 'Down', 700946265), ('vuko', 110, 'N', 'Up', 700946312), ('vuko', 108, 'L', 'Down', 700946453), ('vuko', 108, 'L', 'Up', 700946500), ('vuko', 13, 'Return', 'Down', 700947171)]
{'Input1': '.tie5Roanl'}


ALL CODE BELOW THIS IS FOR TESTING PURPOSES FOR LEARNING THE PYTHON LIBRARIES

In [5]:
####Testing purposes
# This function is called every time a key is pressed
def OnKeyboardEvent(event):
    print('MessageName:',event.MessageName)         # MessageName: key down or key up
    print('Message:',event.Message)                 # Message: 256 -> key down, 257 -> key up
    print('Time:',event.Time)                       # The time at which the event occurred, in milliseconds
    print('Window:',event.Window)                   # A handle to the window in which the event occurred 
    print('WindowName:',event.WindowName)           # The name of the window in which the event occurred i.e. getData.ipynb
    print('Ascii:', event.Ascii, chr(event.Ascii))  # The ASCII value of any character that was pressed i.e. 13 = Enter
    print('Key:', event.Key)                        # The key that was pressed or released i
    print('KeyID:', event.KeyID)                    # The virtual key code of the key that was pressed or released
    print('ScanCode:', event.ScanCode)              # The hardware scan code of the key that was pressed or released
    print('Extended:', event.Extended)              #
    print('Injected:', event.Injected)              # True if the event was injected; otherwise, False
    print('Alt', event.Alt)                         # True if the ALT key is down; otherwise, False
    print('Transition', event.Transition)           # True if the event is from a system-generated event; otherwise, False
    print('---')
    df.loc[len(df)] = [event.Key, event.Ascii, event.Time]

    # Return True to pass the event to other handlers
    # Return False to stop the event from propagating
    return True


In [6]:
# Create a hook manager
hm = pyHook.HookManager()

hm.KeyDown = OnKeyboardEvent
hm.KeyUp = OnKeyboardEvent

# Set the hook
hm.HookKeyboard()

In [7]:
hm.UnhookKeyboard()

In [8]:
import PySimpleGUI as sg

sg.theme("DarkBlue3")
sg.set_options(font=("Courier New", 16))

layout = [
    [sg.Input("Enter to generate an event",     key='Input1')],
    [sg.Input("Enter not to generate an event", key='Input2')],
]
window = sg.Window('Title', layout, finalize=True)
window['Input1'].bind("<Return>", "_Enter")

while True:
    event, values = window.read()
    if event == sg.WINDOW_CLOSED:
        break
    elif event == "Input1" + "_Enter":
        print(event)

window.close()

In [9]:
import PySimpleGUI as sg

layout = [  [sg.Text('Input element that clears after every read')],
            [sg.Input('Initial text', key='-I-', do_not_clear=False)],
            [sg.Button('Go'), sg.Button('Exit')] ]

window = sg.Window('Input auto-clear', layout)

while True:             # Event Loop
    event, values = window.read()
    if event in (None, 'Exit'):
        break
    print(event, values)
window.close()


TASKS
1. Sučelje za prikupit inpute
    a. Osim svojstva pojedine akcije treba spremit i # repeticije riječi -> ne uzimat input za shift
    b. Spremit podatke u neki csv
2. Iz prikupljenih podataka izračunat H, DD, UD,...
3. Iz promjenjenih podataka napravit konačni  csv
    a. Gleda se ukupni input za jednu riječ/lozinku
3. Podijelit konačni csv na brojke i odvojenu u drugi csv koja osoba je to tipkala
4. Podijelit na test i train
5. Istrenirat mrežu
6. Sučelje za test
    a. Upišeš svoj username
    b. Nakon upisa šifre, izbacuje da li si ti ta osoba ili ne